<a href="https://colab.research.google.com/github/mille-s/FORGe_count-rules/blob/main/FORGe_count_rules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Upload a zipped folder with rules and unzip by running this cell
! unzip /content/rule.zip

In [52]:
from posixpath import split
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import codecs
import re
import itertools
import glob
import os

# RE for number of LUs in lexicons: ^\s*"[^"]+_[A-Z]{2}_[0-9]+"

folderPath = '/content/rule'
encoding = 'utf-8'
debug = 'no'

pathOut = '/content/out.txt'

# Delete existing property file
if os.path.exists(pathOut):
  os.remove(pathOut)

list_filepaths = sorted(glob.glob(os.path.join(folderPath, '*.*')))
#V4Design D5.3
language_IDs = ['CA', 'DE', 'EL', 'EN', 'ES', 'GA', 'FR', 'IT', 'PL', 'PT']
print("WARNING! Check that the list of languages is complete: "+str(language_IDs))

rule_all_count = []
rule_con_count = []
rule_agg_count = []
rule_dsynt_count = []
rule_ssynt_count = []
rule_lin_count = []
rule_sent_count = []

def make_stats(list_prefixes_rules, list_prefixes_rules_meta, fo, level):
  num_all = int(len(list_prefixes_rules))
  num_lang_ind = int(list_prefixes_rules.count('GEN'))
  num_lang_spec = int(num_all - num_lang_ind)
  if level == 'all':
    print('# Rules: ' +str(num_all))
    print('# Language-independent rules: ' +str(num_lang_ind))
    print('% Language-independent rules: ' +str(round(100*float(num_lang_ind)/float(num_all), 2)) +'%')
    print('# Language-specific rules: ' +str(num_lang_spec))
    print('% Language-specific rules: '+str(round(100*float(num_lang_spec)/float(num_all), 2)) +'%')
    #print('# Meta rules: ' +str(list_prefixes_rules_meta))
  fo.write('# Rules: '+str(num_all)+'\n')
  fo.write('# Language-independent rules: '+str(num_lang_ind)+'\n')
  fo.write('% Language-independent rules: '+str(round(100*float(num_lang_ind)/float(num_all), 2))+'%\n')
  fo.write('# Language-specific rules: '+str(num_lang_spec)+'\n')
  fo.write('% Language-specific rules: '+str(round(100*float(num_lang_spec)/float(num_all), 2))+'%\n')
    
def fill_lists_rule_count(filename, prefix, list_all, list_con, list_agg, list_dsynt, list_ssynt, list_lin, list_sent):
  list_all.append(prefix)
  if re.search('Con_Sem', filename) or re.search('Sem[^_]*_Sem', filename):
    list_con.append(prefix)
  elif re.search('Con_Agg[1-9]', filename):
    list_agg.append(prefix)
  elif re.search('SemComm_DSynt', filename):
    list_dsynt.append(prefix)
  elif re.search('DSynt_SSynt', filename) or re.search('SSynt_PostProc', filename) or re.search('SSynt_Agg', filename):
    list_ssynt.append(prefix)
  elif re.search('SSynt_DMorph_linearize', filename):
    list_lin.append(prefix)
  elif re.search('S[mM]orph', filename):
    list_sent.append(prefix)

#print('2 Arguments needed: pathInputFolder encoding')
ngrammars = 0
fo = codecs.open(pathOut, 'a', 'utf-8')
for filepath in list_filepaths:
  filename = filepath.rsplit('/', 1)[1]
  if debug == 'yes':
    print(filename)
  if filename.startswith(('1', '2', '3', '4', '5', '6', '7', '8', '9', '0')):
    ngrammars = ngrammars + 1
    fo.write('--------------\n')
    fo.write(filename+'\n')
    fo.write('--------------\n')
    fd = codecs.open(filepath, 'r', encoding)
    rule_grammar_count = []
    rule_meta_count = 0
    lines = fd.readlines()
    # store the language prefix of each rule
    # we start counting at 2 because "leftside" can only happen on line 3 earliest
    x = 2
    while x < len(lines):
      if re.search('^\s*leftside\s*=\s*\[\r*\n', lines[x]):
        if re.search('^[^\]\n]', lines[x+1]):
          # \ufeff is BOM
          if re.search('^\ufeff*\s*[a-zA-Z]+<=>[a-zA-Z]+\s[^\n]+\r*\n', lines[x-2]):
            prefix_languages_rule = []
            rulename = ''
            if re.search(' : ', lines[x-2]):
              rulename = lines[x-2].split(' : ')[0].split(' ', 1)[1]
            elif  re.search('\r\n', lines[x-2]):
              rulename = lines[x-2].split('\r\n')[0].split(' ', 1)[1]
            elif  re.search('\n', lines[x-2]):
              rulename = lines[x-2].split('\n')[0].split(' ', 1)[1]
            # Find all consecutive language prefixes at the beginning of the rulename
            split_rulename = ''
            if re.search('_', rulename):
              split_rulename = rulename.split('_')
            else:
              split_rulename = [rulename]
            c = 0
            while c < 1000:
              if c < len(split_rulename):
                language_prefix = split_rulename[c]
                if language_prefix in language_IDs:
                  prefix_languages_rule.append(language_prefix)
                  c = c + 1
                else:
                  c = c + 1000
              else:
                c = c + 1000
            if len(prefix_languages_rule) > 0:
              whole_language_prefix = '_'.join(prefix_languages_rule)
              if debug == 'yes':
                print(whole_language_prefix)
              fill_lists_rule_count(filename, whole_language_prefix, rule_all_count, rule_con_count, rule_agg_count, rule_dsynt_count, rule_ssynt_count, rule_lin_count, rule_sent_count)
              rule_grammar_count.append(whole_language_prefix)
            else:
              rule_grammar_count.append('GEN')
              fill_lists_rule_count(filename, 'GEN', rule_all_count, rule_con_count, rule_agg_count, rule_dsynt_count, rule_ssynt_count, rule_lin_count, rule_sent_count)
            x = x + 1
          else:
            x = x + 1
        else:
          rule_meta_count = rule_meta_count + 1
          x = x + 1
      else:
        x = x + 1
      
    if len(rule_grammar_count) > 0:
      make_stats(rule_grammar_count, rule_meta_count, fo, str(filename))
  else:
    pass

dot = '--------------\n'
noRule = 'No rules found!\n'
fo.write(dot)
allRules = '\n\nAll rulesets ('+str(ngrammars)+' grammars)'
print(allRules+'--------------')
fo.write(allRules+'\n')
fo.write(dot)
if len(rule_all_count) > 0:
    make_stats(rule_all_count, '', fo, 'all')
else:
    print(noRule)
fo.write(dot)
fo.write('Con rulesets\n')
fo.write(dot)
if len(rule_con_count) > 0:
    make_stats(rule_con_count, '', fo, 'con')
else:
    print(noRule)
    fo.write(noRule)
fo.write(dot)
fo.write('Agg rulesets\n')
fo.write(dot)
if len(rule_agg_count) > 0:
    make_stats(rule_agg_count, '', fo, 'agg')
else:
    print(noRule)
    fo.write(noRule)
fo.write(dot)
fo.write('DSynt rulesets\n')
fo.write(dot)
if len(rule_dsynt_count) > 0:
    make_stats(rule_dsynt_count, '', fo, 'dsynt')
else:
    print(noRule)
    fo.write(noRule)
fo.write(dot)
fo.write('SSynt rulesets\n')
fo.write(dot)
if len(rule_ssynt_count) > 0:
    make_stats(rule_ssynt_count, '', fo, 'ssynt')
else:
    print(noRule)
    fo.write(noRule)
fo.write(dot)
fo.write('Lin rulesets\n')
fo.write(dot)
if len(rule_lin_count) > 0:
    make_stats(rule_lin_count, '', fo, 'lin')
else:
    print(noRule)
    fo.write(noRule)
fo.write(dot)
fo.write('Sent rulesets\n')
fo.write(dot)
if len(rule_sent_count) > 0:
    make_stats(rule_sent_count, '', fo, 'sent')
else:
    print(noRule)
    fo.write(noRule)
fo.close()


WARNING! Check that the list of languages is complete: ['CA', 'DE', 'EL', 'EN', 'ES', 'GA', 'FR', 'IT', 'PL', 'PT']


All rulesets (16 grammars)--------------
# Rules: 2497
# Language-independent rules: 1870
% Language-independent rules: 74.89%
# Language-specific rules: 627
% Language-specific rules: 25.11%
